In [1]:
import os
from sklearn.datasets import load_iris
import pyspark.pandas as ps
import pandas as pd
import pyspark
from pyspark.sql import SparkSession

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# spark = SparkSession.builder.appName("Iris").getOrCreate()
spark = SparkSession.builder.appName("grow").master("spark://172.18.0.3:7077").getOrCreate()
sc = pyspark.SparkConf()
spark.version

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/19 16:36:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'3.3.0'

In [2]:
X, y = load_iris(return_X_y=True, as_frame= True)
df = X
df['target'] = y
df = spark.createDataFrame(df)
df.show(10)

+-----------------+----------------+-----------------+----------------+------+
|sepal length (cm)|sepal width (cm)|petal length (cm)|petal width (cm)|target|
+-----------------+----------------+-----------------+----------------+------+
|              5.1|             3.5|              1.4|             0.2|     0|
|              4.9|             3.0|              1.4|             0.2|     0|
|              4.7|             3.2|              1.3|             0.2|     0|
|              4.6|             3.1|              1.5|             0.2|     0|
|              5.0|             3.6|              1.4|             0.2|     0|
|              5.4|             3.9|              1.7|             0.4|     0|
|              4.6|             3.4|              1.4|             0.3|     0|
|              5.0|             3.4|              1.5|             0.2|     0|
|              4.4|             2.9|              1.4|             0.2|     0|
|              4.9|             3.1|              1.

In [ ]:
# vectorize all numerical columns into a single feature column
feature_cols = df.columns[:-1]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
data = assembler.transform(df)
data.show(5)

In [ ]:
# convert text labels into indices
data = data.select(['features', 'target'])
label_indexer = pyspark.ml.feature.StringIndexer(inputCol='target', outputCol='label').fit(data)
data = label_indexer.transform(data)
data.show(5)


In [ ]:
# only select the features and label column
data = data.select(['features', 'label'])
print("Reading for machine learning")
data.show(5)

In [ ]:
train, test = data.randomSplit([0.70, 0.30])

In [ ]:
from pyspark.ml.classification import LogisticRegression

pipe = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

pipe = pipe.fit(train)

In [ ]:
prediction = pipe.transform(test)
prediction

In [ ]:
evaluator = pyspark.ml.evaluation.MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = evaluator.evaluate(prediction)
accuracy